#Added Chris suggestion from this discussion : https://www.kaggle.com/c/tensorflow-great-barrier-reef/discussion/307668#1692150

Public : 0.689  Private : 0.685

##### YOLOv5 detections + TRACKING submission made on COTS dataset

Tracking is a good idea, and we got it from Aleksandr Snorkin's notebook. Thanks Aleksandr Snorkin!

LB:  Scoring...

If you like this work, please upvote !

References for this notebook are listed below:

[1][YoloX inference + Tracking on COTS [LB 0.539]](https://www.kaggle.com/parapapapam/yolox-inference-tracking-on-cots-lb-0-539)

[2][higher resolution and confidence](https://www.kaggle.com/macxiao/higher-resolution-and-confidence)

[3][Yolov5 is all you need](https://www.kaggle.com/steamedsheep/yolov5-is-all-you-need),

please upvote them also! 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from tqdm import tqdm
import sys

sys.path.append('../input/tensorflow-great-barrier-reef')

In [ ]:
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [ ]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

In [ ]:
model = torch.hub.load('../input/yolov5-lib-ds', 
                       'custom', 
                       path='../input/yolov5s6/f2_sub2.pt',
                       source='local',
                       force_reload=True)  # local repo
model.conf = 0.20

In [ ]:
# norfair dependencies
%cd /kaggle/input/norfair031py3/
!pip install commonmark-0.9.1-py2.py3-none-any.whl -f ./ --no-index
!pip install rich-9.13.0-py3-none-any.whl

!mkdir /kaggle/working/tmp
!cp -r /kaggle/input/norfair031py3/filterpy-1.4.5/filterpy-1.4.5/ /kaggle/working/tmp/
%cd /kaggle/working/tmp/filterpy-1.4.5/
!pip install .
!rm -rf /kaggle/working/tmp

# norfair
%cd /kaggle/input/norfair031py3/
!pip install norfair-0.3.1-py3-none-any.whl -f ./ --no-index
%cd /kaggle/working/

In [ ]:
##############################################################
#                      Tracking helpers                      #
##############################################################

import numpy as np
from norfair import Detection, Tracker

# Helper to convert bbox in format [x_min, y_min, x_max, y_max, score] to norfair.Detection class
def to_norfair(detects, frame_id):
    result = []
    for x_min, y_min, x_max, y_max, score in detects:
        xc, yc = (x_min + x_max) / 2, (y_min + y_max) / 2
        w, h = x_max - x_min, y_max - y_min
        result.append(Detection(points=np.array([xc, yc]), scores=np.array([score]), data=np.array([w, h, frame_id])))
        
    return result

# Euclidean distance function to match detections on this frame with tracked_objects from previous frames
def euclidean_distance(detection, tracked_object):
    return np.linalg.norm(detection.points - tracked_object.estimate)

In [ ]:
tracker = Tracker(
    distance_function=euclidean_distance, 
    distance_threshold=30,
    hit_inertia_min=3,
    hit_inertia_max=6,
    initialization_delay=1,
)
frame_id = 0

In [ ]:
for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
    detects = []
    anno = ''
    r = model(img, size=6400, augment=True)
    if r.pandas().xyxy[0].shape[0] == 0:
        anno = ''
    else:
        for idx, row in r.pandas().xyxy[0].iterrows():
            area = (int(row.xmax) - int(row.xmin)) * (int(row.ymax) - int(row.ymin))
            if row.confidence > 0.53 and area > 270:     
                anno += '{} {} {} {} {} '.format(row.confidence, int(row.xmin), int(row.ymin), int(row.xmax-row.xmin), int(row.ymax-row.ymin))
                detects.append([int(row.xmin), int(row.ymin), int(row.xmin)+int(row.xmax-row.xmin), int(row.ymin)+int(row.ymax-row.ymin), row.confidence])

    tracked_objects = tracker.update(detections=to_norfair(detects, frame_id))
    for tobj in tracked_objects:
        bbox_width, bbox_height, last_detected_frame_id = tobj.last_detection.data
        if last_detected_frame_id == frame_id:  # Skip objects that were detected on current frame
            continue
        if last_detected_frame_id <= frame_id-2:  
            continue
            
        # Add objects that have no detections on current frame to predictions
        xc, yc = tobj.estimate[0]
        x_min, y_min = int(round(xc - bbox_width / 2)), int(round(yc - bbox_height / 2))
        score = tobj.last_detection.scores[0]
        anno += '{} {} {} {} {} '.format(score, x_min, y_min, bbox_width, bbox_height)
        
    pred_df['annotations'] = anno.strip(' ')
    env.predict(pred_df)
    frame_id += 1